In [5]:
import pandas as pd
import collections
import numpy as np
from math import ceil

In [6]:
# Opciones de pandas
rows, cols = 400, 100
pd.set_option("display.max.columns", cols)
pd.set_option("display.max.rows", rows)
# Rutas
PATH = "../../data-article/fundamental_data/"

## 1. Dataset Configuration

In [3]:
# Generar datos de cierre de cada stock en cada trimestre
path = '../../data-article/technical_data/daily/'
files = ['PEP', 'IBM', 'BDX', 'NSC', 'GL']
for file in files:
    file += '.csv'
    print("Processing stock:", file)
    data = pd.read_csv(path+file, header=0)
    data.set_index(pd.DatetimeIndex(data['Date']), inplace=True)
    data.drop(columns=['Date'], inplace=True)
    df = pd.DataFrame()
    for year in set(data.index.year):
        months = [3,6,9,12]
        for month in months:
            try:
                df = pd.concat([df, data[(data.index.month==month) & (data.index.year==year)].iloc[-1]], axis=1)
            except:
                pass
    df = df.T
    df.to_csv('../../data-article/technical_data/quarterly/{}'.format(file))

Processing stock: PEP.csv
Processing stock: IBM.csv
Processing stock: BDX.csv
Processing stock: NSC.csv
Processing stock: GL.csv


In [16]:
# Generar dataset de acciones y atributos completo

# Importar librerías
import pandas as pd
from os import listdir

# Variables importantes
stock_list = []

# Unir income, balance y cash
path = PATH + 'csvs/'
path_tech = '../../data-article/technical_data/quarterly/'
stocks = listdir(path)
all_data = pd.DataFrame()

#for stock in stocks:
for stock in ['PEP', 'IBM', 'BDX', 'NSC', 'GL']:
    
    file = stock+'.csv'
    df = pd.read_csv(path+file, header=0, index_col=0)
    df.set_index(pd.DatetimeIndex(list(df.index)), inplace=True)
    df.index = df.index.strftime('%Y-%m')
    
    # Los datos deben ir ordenados de manera descendente
    df.sort_index(axis=0, ascending=False, inplace=True)
    
    # Poner todas las unidades en Billones Americanos (ANTES)
    """d = {'M': '*0.001', 'B': '', 'T': '*1000'}
    df = df.replace(d, regex=True)
    df = df.applymap(pd.eval, na_action='ignore')
    """
    # Poner todas las unidades en Millones
    # Los datos ya están en millones
    
    # Crear nuevos atributos necesarios para realizar cálculos en ratios y demás indicadores
    netcashop_ttm, netcapex_ttm, netincome_ttm = [None for _ in range(len(df.index))], [None for _ in range(len(df.index))], [None for _ in range(len(df.index))]
    try:
        for i in range(len(df.index)-4, -1, -1):
                netcashop_ttm[i] = df['CashFromOperations'][i] + df['CashFromOperations'][i+1] + df['CashFromOperations'][i+2] + df['CashFromOperations'][i+3]
                netcapex_ttm[i] = df['NetCapitalExpenditures'][i] + df['NetCapitalExpenditures'][i+1] + df['NetCapitalExpenditures'][i+2] + df['NetCapitalExpenditures'][i+3]
                netincome_ttm[i] = df['NetIncome'][i] + df['NetIncome'][i+1] + df['NetIncome'][i+2] + df['NetIncome'][i+3]
        df['CashfromOperations(TTM)'] = netcashop_ttm
        df['NetCapitalExpenditures(TTM)'] = netcapex_ttm
        df['NetIncome(TTM)'] = netincome_ttm
    except:
        continue
    
    # Añadir la colomnua de precios (Close adj) al dataset
    data = pd.read_csv(path_tech+stock+'.csv',header=0, index_col=0)
    data.set_index(pd.DatetimeIndex(list(data.index)), inplace=True)
    data.index = data.index.strftime('%Y-%m')
    df = pd.concat([df,data['Close']], axis=1).sort_index(axis=0, ascending=False)
    df.rename(columns={'Close':'PricePerShare'}, inplace=True)

    # Calcular y agregar ratios e indicadores
    roe = [None for _ in range(len(df.index))]
    for i in range(len(df.index)-6, -1, -1):
        bv_act = (df['TotalAssets'][i] - df['TotalLiabilities'][i])
        bv_past = (df['TotalAssets'][i+5] - df['TotalLiabilities'][i+5])
        try:
            roe[i] = df['NetIncome(TTM)'][i] / ((bv_act + bv_past) / 2) * 100
        except:
            roe[i] = None
    try:
        df['FCF'] = df.apply(lambda col: col['CashfromOperations(TTM)'] + col['NetCapitalExpenditures(TTM)'], axis=1)
    except:
        continue
    df['ROE'] = roe

    # Eliminar todos los atributos que no se encuentran en la lista de incluídos
    #included = ['Revenue','GrossProfit','SG&AExpense','ResearchandDevelopmentExpense','NetOperatingInterestIncome','TotalOperatingExpenses','OperatingIncome','NetNon-OperatingInterestIncomeExpense','OtherIncomeandExpenses','NetInterestIncome','IncomefromContinuingOperations','NetIncome','EBIT','EPSBasic','EPSDiluted','AverageBasicSharesOutstanding','DividendPerShare','Cash','CashandEquivalents','ShortTermInvestments','TotalReceivables','Inventories','TotalCurrentAssets','LandandImprovements','NetPP&E','TotalLongTermAssets','TotalAssets','PayablesandAccruedExpenses','CurrentDebt&CapitalLeaseObligation','CurrentDeferredRevenue','CurrentDeferredLiabilities','TotalCurrentLiabilities','TotalLongTermLiabilities','TotalLiabilities','TotalCapitalStock','RetainedEarnings','PreferredStock','ShareholdersEquity','NetIncome','TotalDepreciation,Amortization,Depletion','DeferredTaxes','StockBasedCompensation','OtherNoncashItems','CashfromOperations','SaleofPPE','NetChangeinPP&E','NetDivestitures(Acquisitions)','TotalNetChangeinInvestments','CashfromInvesting','NetChangeinLongTermDebt','NetChangeinShortTermDebt','NetDebtIssuance','TotalDividendsPaid','CashfromFinancing','BeginningCash','ChangeinCash','EndingCash','IssuanceofDebt','DebtRepayment','RepurchaseofCapitalStock','EPSDiluted(TTM)','CashfromOperations(TTM)','NetChangeinCapitalExpenditures(TTM)','Revenue(TTM)','P/E','PEG','FCF','P/FCF','P/S','P/B','ROE','PricePerShare']
    #drops = list( set(df.columns) - set(included))
    #df.drop(drops, axis=1, inplace=True)
    
    df.columns = pd.MultiIndex.from_product([df.columns, [stock]], names=['Attributes', 'Symbol'])
    all_data = pd.concat([all_data, df], axis=1).sort_index(axis=1)
    print('Procesando stock:',stock)
    stock_list.append(stock)
#date_drops = list( set(all_data.index) - set(date_include) )
#all_data.drop(date_drops, axis=0, errors='ignore', inplace=True)
all_data.to_csv(PATH+'backup/complete_dataset.csv')
print(stock_list)
print(len(stock_list))

Procesando stock: PEP
Procesando stock: IBM
Procesando stock: BDX
Procesando stock: NSC
Procesando stock: GL
['PEP', 'IBM', 'BDX', 'NSC', 'GL']
5


In [17]:
data = pd.read_csv(PATH+'backup/complete_dataset.csv', header=[0,1], index_col=0, low_memory=False)
data.set_index(pd.DatetimeIndex(list(data.index)).strftime('%Y-%m'), inplace=True)
data

Attributes AverageBasicSharesOutstanding                                 \
Symbol                               BDX        GL        IBM       NSC   
2021-03                         291.0950  103.4829   893.6000  251.4000   
2020-12                         290.5900  105.7780   892.6000  253.2075   
2020-09                         290.3480  106.1468   891.4000  254.6000   
2020-06                         282.3850  106.4413   889.4000  255.4000   
2020-03                         272.0140  107.2852   888.0000  257.3000   
2019-12                         271.1020  108.1277   887.1000  261.6000   
2019-09                         270.6100  108.8134   886.0000  262.1000   
2019-06                         270.2490  109.6497   886.3000  264.8000   
2019-03                         269.8820  110.3015   889.6000  267.1000   
2018-12                         269.0350  111.2953   901.3000  271.0425   
2018-09                         268.2010  112.5876   911.2000  275.5000   
2018-06                         267.8360  113.3502   915.1000  281.3000   
2018-03                         267.3410  114.2105   920.7000  283.5000   
2017-12                         230.0380  115.0056   924.5000  287.8000   
2017-09                         228.2180  115.9211   929.4000  287.1000   
2017-06                         220.8070  116.6467   934.9000  289.0000   
2017-03                         213.5830  117.7705   942.4000  290.3000   
2016-12                         213.0640  118.5634   948.6000  291.2000   
2016-09                         213.5710  119.4806   954.0000  292.7000   
2016-06                         213.0830  120.4799   957.4000  294.7000   
2016-03                         212.4690  121.4808   961.7000  297.2000   
2015-12                         211.6890  123.1481   969.4000  297.9000   
2015-09                         210.9850  124.4580   975.1000  300.1000   
2015-06                         210.1750  125.8169   982.3000  302.9000   
2015-03                         196.0850  127.1211   988.1000  306.8000   
2014-12                         192.8440  128.3240   990.4000  309.0000   
2014-09                         192.3320  129.9699   991.8000  309.4000   
2014-06                         193.0540  131.4912   999.6000  309.5000   
2014-03                         193.6090  133.1635  1035.2000  309.5000   
2013-12                         194.2030  134.7330  1072.5000  308.7000   
2013-09                         194.6970  136.9038  1090.9000  310.4000   
2013-06                         194.8790  138.2931  1100.9000  314.1000   
2013-03                         194.6090  140.3712  1113.7000  314.5000   
2012-12                         196.4270  141.5503  1124.7000  314.8000   
2012-09                         199.0720  142.6888  1137.2000  317.7000   
2012-06                         202.0150  145.3445  1149.0000  322.7000   
2012-03                         206.4260  150.1025  1159.1000  328.3000   
2011-12                         214.3000  153.5535  1172.2000  332.8000   
2011-09                         216.7260  157.1691  1188.6000  343.2000   
2011-06                         218.9660  166.5152  1204.8000  351.0000   
2011-03                         220.8940  262.8390  1222.2000  355.2000   
2010-12                         228.0830  179.4183  1240.1000  360.7000   
2010-09                         231.3960  272.6180  1255.2000  366.3000   
2010-06                         233.2420  276.8079  1278.6000  369.7000   
2010-03                         235.3250  186.4043  1301.2000  369.5000   
2009-12                         237.3600  186.2782  1318.4000  367.9000   
2009-09                         239.1620  186.1790  1319.9000  367.3000   
2009-06                         240.1090  186.1532  1326.1000  366.8000   
2009-03                         240.2390  188.7197  1344.3000  366.2000   
2008-12                         242.3970  191.1187  1339.1000  365.8000   
2008-09                         243.8630  195.7932  1350.7000  372.5000   
2008-06                  